In [5]:
import numpy as np
import matplotlib as plt
import pandas as pd
import os
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from Model import IMVTensorLSTM
import torch
print(torch.cuda.device_count())
print(torch.cuda.is_available())
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

0
False


In [37]:
data = pd.read_csv(r"E:\code\Battery\data\battery_features_norm_10_1525.csv")
## 第一组
# train_bound = 857
# val_bound = 1730
# test_bound_1 = 2594
# test_bound_2 = 3368
# test_bound_3 = 4450
# Index = [0,2,6,8,10,11]
## 第二组
# train_bound = 761
# val_bound = 1504
# test_bound_1 = 2411
# test_bound_2 = 3159
# test_bound_3 = 3775
# Index = [0,1,2,6,8,10]
## 第三组
# train_bound = 778
# val_bound = 1485
# test_bound_1 = 2093
# test_bound_2 = 2769
# test_bound_3 = 3508
# Index = [2,6,0,10,8,11]
## 第四组
# train_bound = 657
# val_bound = 1187
# test_bound_1 = 1859
# test_bound_2 = 2360
# test_bound_3 = 2920
# Index = [2,0,6,10,8,5]
## 第五组
# train_bound = 914
# val_bound = 1592
# test_bound_1 = 2581
# test_bound_2 = 3670
# test_bound_3 = 4421
# Index = [2,0,6,7,11,10]
## 第六组
# train_bound = 443
# val_bound = 897
# test_bound_1 = 1374
# test_bound_2 = 1880
# test_bound_3 = 2480
# Index = [0,2,4,5,6,11]
## 第七组
# train_bound = 819
# val_bound = 1591
# test_bound_1 = 2630
# test_bound_2 = 3459
# test_bound_3 = 4402
# Index = [2,9,0,6,8,11]
## 第八组
# train_bound = 910
# val_bound = 1738
# test_bound_1 = 2730
# test_bound_2 = 3485
# test_bound_3 = 4349
# # Index = [2,0,6,11,8,10]
# Index = [2,0,6,9,8,10]
## 第九组
# train_bound = 888
# val_bound = 1780
# test_bound_1 = 2946
# test_bound_2 = 3783
# test_bound_3 = 4558
# # Index = [2,6,0,8,7,15]
# Index = [0,2,6,7,8,11]
## 第十组
train_bound = 532
val_bound = 987
test_bound_1 = 1533
test_bound_2 = 2332
test_bound_3 = 2898
test_bound_4 = 3261
test_bound_5 = 3690
test_bound_6 = 4110
Index = [1,2,7,8,10,11]

In [38]:
data.head()
cols = list(data.columns[1:])
cols = [cols[Index[0]], cols[Index[1]], cols[Index[2]], cols[Index[3]], cols[Index[4]], cols[Index[5]]]

In [40]:
cols
data['capacity'] = data['capacity'].ffill().bfill()
depth = 5

In [42]:
My_data = data[:train_bound]
X = np.zeros((len(My_data), depth, len(cols)))
for i, name in enumerate(cols):
    for j in range(depth):
        X[:, j, i] = My_data[name].shift(depth - j - 1).bfill()
X_train = X

My_data = data[train_bound:val_bound]
X = np.zeros((len(My_data), depth, len(cols)))
for i, name in enumerate(cols):
    for j in range(depth):
        X[:, j, i] = My_data[name].shift(depth - j - 1).bfill()
X_val = X

My_data = data[val_bound:test_bound_1]
X = np.zeros((len(My_data), depth, len(cols)))
for i, name in enumerate(cols):
    for j in range(depth):
        X[:, j, i] = My_data[name].shift(depth - j - 1).bfill()
X_test_1 = X

My_data = data[test_bound_1:test_bound_2]
X = np.zeros((len(My_data), depth, len(cols)))
for i, name in enumerate(cols):
    for j in range(depth):
        X[:, j, i] = My_data[name].shift(depth - j - 1).bfill()
X_test_2 = X

My_data = data[test_bound_2:test_bound_3]
X = np.zeros((len(My_data), depth, len(cols)))
for i, name in enumerate(cols):
    for j in range(depth):
        X[:, j, i] = My_data[name].shift(depth - j - 1).bfill()
X_test_3 = X


## SCU数据集专有
My_data = data[test_bound_3:test_bound_4]
X = np.zeros((len(My_data), depth, len(cols)))
for i, name in enumerate(cols):
    for j in range(depth):
        X[:, j, i] = My_data[name].shift(depth - j - 1).bfill()
X_test_4 = X

My_data = data[test_bound_4:test_bound_5]
X = np.zeros((len(My_data), depth, len(cols)))
for i, name in enumerate(cols):
    for j in range(depth):
        X[:, j, i] = My_data[name].shift(depth - j - 1).bfill()
X_test_5 = X

My_data = data[test_bound_5:test_bound_6]
X = np.zeros((len(My_data), depth, len(cols)))
for i, name in enumerate(cols):
    for j in range(depth):
        X[:, j, i] = My_data[name].shift(depth - j - 1).bfill()
X_test_6 = X

In [44]:
y = data['capacity'].shift(0).ffill().values

In [45]:
# X_train = X[:train_bound]
# X_val = X[train_bound:val_bound]
# X_test_1 = X[val_bound:test_bound_1]
# X_test_2 = X[test_bound_1:test_bound_2]
# X_test_3 = X[test_bound_2:test_bound_3]

y_train  = y[:train_bound]
y_val    = y[train_bound:val_bound]
y_test_1 = y[val_bound:test_bound_1]
y_test_2 = y[test_bound_1:test_bound_2]
y_test_3 = y[test_bound_2:test_bound_3]

## SCU数据集专有
y_test_4 = y[test_bound_3:test_bound_4]
y_test_5 = y[test_bound_4:test_bound_5]
y_test_6 = y[test_bound_5:test_bound_6]

In [46]:
## 不需要重新归一化
X_train_min = 0
X_train_max = 1
y_train_min = 0
y_train_max = 1

## 重新归一化
# X_train_min, X_train_max = X_train.min(axis=0), X_train.max(axis=0)
# y_train_min, y_train_max = y_train.min(axis=0), y_train.max(axis=0)

In [47]:
X_train = (X_train - X_train_min)/(X_train_max - X_train_min + 1e-9)
X_val = (X_val - X_train_min)/(X_train_max - X_train_min + 1e-9)
X_test_1 = (X_test_1 - X_train_min)/(X_train_max - X_train_min + 1e-9)
X_test_2 = (X_test_2 - X_train_min)/(X_train_max - X_train_min + 1e-9)
X_test_3 = (X_test_3 - X_train_min)/(X_train_max - X_train_min + 1e-9)

y_train = (y_train - y_train_min)/(y_train_max - y_train_min +1e-9)
y_val = (y_val - y_train_min)/(y_train_max - y_train_min + 1e-9)
y_test_1 = (y_test_1 - y_train_min)/(y_train_max - y_train_min + 1e-9)
y_test_2 = (y_test_2 - y_train_min)/(y_train_max - y_train_min + 1e-9)
y_test_3 = (y_test_3 - y_train_min)/(y_train_max - y_train_min + 1e-9)

## SCU数据集专有
X_test_4 = (X_test_4 - X_train_min)/(X_train_max - X_train_min + 1e-9)
X_test_5 = (X_test_5 - X_train_min)/(X_train_max - X_train_min + 1e-9)
X_test_6 = (X_test_6 - X_train_min)/(X_train_max - X_train_min + 1e-9)

y_test_4 = (y_test_4 - y_train_min)/(y_train_max - y_train_min + 1e-9)
y_test_5 = (y_test_5 - y_train_min)/(y_train_max - y_train_min + 1e-9)
y_test_6 = (y_test_6 - y_train_min)/(y_train_max - y_train_min + 1e-9)

In [48]:
X_train_t = torch.Tensor(X_train)
X_val_t = torch.Tensor(X_val)
X_test_t_1 = torch.Tensor(X_test_1)
X_test_t_2 = torch.Tensor(X_test_2)
X_test_t_3 = torch.Tensor(X_test_3)

y_train_t = torch.Tensor(y_train)
y_val_t = torch.Tensor(y_val)
y_test_t_1 = torch.Tensor(y_test_1)
y_test_t_2 = torch.Tensor(y_test_2)
y_test_t_3 = torch.Tensor(y_test_3)

## SCU数据集专有
X_test_t_4 = torch.Tensor(X_test_4)
X_test_t_5 = torch.Tensor(X_test_5)
X_test_t_6 = torch.Tensor(X_test_6)

y_test_t_4 = torch.Tensor(y_test_4)
y_test_t_5 = torch.Tensor(y_test_5)
y_test_t_6 = torch.Tensor(y_test_6)

In [49]:
train_loader  = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=64, shuffle=True)
My_train_loader= DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=64, shuffle=False)
val_loader    = DataLoader(TensorDataset(X_val_t, y_val_t), batch_size=64, shuffle=False)
test_loader_1 = DataLoader(TensorDataset(X_test_t_1, y_test_t_1), batch_size=64, shuffle=False)
test_loader_2 = DataLoader(TensorDataset(X_test_t_2, y_test_t_2), batch_size=64, shuffle=False)
test_loader_3 = DataLoader(TensorDataset(X_test_t_3, y_test_t_3), batch_size=64, shuffle=False)

## SCU数据集专有
test_loader_4 = DataLoader(TensorDataset(X_test_t_4, y_test_t_4), batch_size=64, shuffle=False)
test_loader_5 = DataLoader(TensorDataset(X_test_t_5, y_test_t_5), batch_size=64, shuffle=False)
test_loader_6 = DataLoader(TensorDataset(X_test_t_6, y_test_t_6), batch_size=64, shuffle=False)

In [51]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = IMVTensorLSTM(X_train_t.shape[2], 1, 512).to(device)

opt = torch.optim.Adam(model.parameters(), lr=0.001)
epoch_scheduler = torch.optim.lr_scheduler.StepLR(opt, 20, gamma=0.9)

In [52]:
epochs = 100
loss = nn.MSELoss()
patience = 35
min_val_loss = 9999
counter = 0
for i in range(epochs):
    mse_train = 0
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        opt.zero_grad()
        y_pred, alphas, betas = model(batch_x)
        y_pred = y_pred.squeeze(1)
        l = loss(y_pred, batch_y)
        l.backward()
        mse_train += l.item()*batch_x.shape[0]
        opt.step()
    epoch_scheduler.step()
    with torch.no_grad():
        mse_val = 0
        preds = []
        true = []
        for batch_x, batch_y in My_train_loader:
            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()
            output, alphas, betas = model(batch_x)
            output = output.squeeze(1)
            preds.append(output.detach().cpu().numpy())
            true.append(batch_y.detach().cpu().numpy())
            mse_val += loss(output, batch_y).item()*batch_x.shape[0]
    preds = np.concatenate(preds)
    true = np.concatenate(true)
    
    if min_val_loss > mse_val**0.5:
        min_val_loss = mse_val**0.5
        print("Saving...")
        torch.save(model.state_dict(), "imv_tensor_lstm_soh.pt")
        counter = 0
    else: 
        counter += 1
    
    # if counter == patience:
    #     break
    print("Iter: ", i, "train: ", (mse_train/len(X_train_t))**0.5, "val: ", (mse_val/len(X_val_t))**0.5)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
model.load_state_dict(torch.load("imv_tensor_lstm_soh.pt"))

In [12]:
## 训练样本
with torch.no_grad():
    mse_val = 0
    preds = []
    true = []
    alphas = []
    betas = []
    for batch_x, batch_y in My_train_loader:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        output, a, b = model(batch_x)
        output = output.squeeze(1)
        preds.append(output.detach().cpu().numpy())
        true.append(batch_y.detach().cpu().numpy())
        alphas.append(a.detach().cpu().numpy())
        betas.append(b.detach().cpu().numpy())
        mse_val += loss(output, batch_y).item()*batch_x.shape[0]
preds = np.concatenate(preds)
true = np.concatenate(true)

NameError: name 'My_train_loader' is not defined

In [ ]:
preds = (preds*(y_train_max - y_train_min) + y_train_min)*0.2+0.8
true = ((true*(y_train_max - y_train_min) + y_train_min))*0.2+0.8
mae = mean_absolute_error(true, preds)*100
mape = np.mean(np.abs((true - preds)/true))*100
rmse = sqrt(mean_squared_error(true, preds))*100
R2 = r2_score(true, preds)*100
mae, mape, rmse, R2

In [13]:
plt.figure(figsize=(10, 5))
plt.plot(preds)
plt.plot(true)
plt.show()

AttributeError: module 'matplotlib' has no attribute 'figure'

In [ ]:
mae = np.zeros(7)
mape = np.zeros(7)
rmse = np.zeros(7)
R2 = np.zeros(7)

In [14]:
## 验证样本
with torch.no_grad():
    mse_val = 0
    preds = []
    true = []
    alphas = []
    betas = []
    for batch_x, batch_y in val_loader:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        output, a, b = model(batch_x)
        output = output.squeeze(1)
        preds.append(output.detach().cpu().numpy())
        true.append(batch_y.detach().cpu().numpy())
        alphas.append(a.detach().cpu().numpy())
        betas.append(b.detach().cpu().numpy())
        mse_val += loss(output, batch_y).item()*batch_x.shape[0]
preds = np.concatenate(preds)
true = np.concatenate(true)

NameError: name 'val_loader' is not defined

In [ ]:
preds = (preds * (y_train_max - y_train_min) + y_train_min) * 0.2 + 0.8
true = ((true * (y_train_max - y_train_min) + y_train_min)) * 0.2 + 0.8
mae[0] = mean_absolute_error(true, preds) * 100
mape[0] = np.mean(np.abs((true - preds) / true)) * 100
rmse[0] = sqrt(mean_squared_error(true, preds)) * 100
R2[0] = r2_score(true, preds) * 100
mae[0], mape[0], rmse[0], R2[0]

In [15]:
# np.savetxt("preds_01_01.txt",preds)
# np.savetxt("true_01_01.txt",true)

In [16]:
plt.figure(figsize=(10, 5))
plt.plot(preds)
plt.plot(true)
plt.show()

AttributeError: module 'matplotlib' has no attribute 'figure'

In [17]:
## 第一个测试样本
with torch.no_grad():
    mse_val = 0
    preds = []
    true = []
    alphas = []
    betas = []
    for batch_x, batch_y in test_loader_1:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        output, a, b = model(batch_x)
        output = output.squeeze(1)
        preds.append(output.detach().cpu().numpy())
        true.append(batch_y.detach().cpu().numpy())
        alphas.append(a.detach().cpu().numpy())
        betas.append(b.detach().cpu().numpy())
        mse_val += loss(output, batch_y).item() * batch_x.shape[0]
preds = np.concatenate(preds)
true = np.concatenate(true)

NameError: name 'test_loader_1' is not defined

In [ ]:
preds = (preds * (y_train_max - y_train_min) + y_train_min) * 0.2 + 0.8
true = ((true * (y_train_max - y_train_min) + y_train_min)) * 0.2 + 0.8
mae[1] = mean_absolute_error(true, preds) * 100
mape[1] = np.mean(np.abs((true - preds) / true)) * 100
rmse[1] = sqrt(mean_squared_error(true, preds)) * 100
R2[1] = r2_score(true, preds) * 100
mae[1], mape[1], rmse[1], R2[1]

In [18]:
# np.savetxt("preds_01_02.txt",preds)
# np.savetxt("true_01_02.txt",true)

In [19]:
plt.figure(figsize=(10, 5))
plt.plot(preds)
plt.plot(true)
plt.show()

AttributeError: module 'matplotlib' has no attribute 'figure'

In [20]:
## 第二个测试样本
with torch.no_grad():
    mse_val = 0
    preds = []
    true = []
    alphas = []
    betas = []
    for batch_x, batch_y in test_loader_2:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        output, a, b = model(batch_x)
        output = output.squeeze(1)
        preds.append(output.detach().cpu().numpy())
        true.append(batch_y.detach().cpu().numpy())
        alphas.append(a.detach().cpu().numpy())
        betas.append(b.detach().cpu().numpy())
        mse_val += loss(output, batch_y).item() * batch_x.shape[0]
preds = np.concatenate(preds)
true = np.concatenate(true)

NameError: name 'test_loader_2' is not defined

In [ ]:
preds = (preds * (y_train_max - y_train_min) + y_train_min) * 0.2 + 0.8
true = ((true * (y_train_max - y_train_min) + y_train_min)) * 0.2 + 0.8
mae[2] = mean_absolute_error(true, preds) * 100
mape[2] = np.mean(np.abs((true - preds) / true)) * 100
rmse[2] = sqrt(mean_squared_error(true, preds)) * 100
R2[2] = r2_score(true, preds) * 100
mae[2], mape[2], rmse[2], R2[2]

In [21]:
# np.savetxt("preds_01_03.txt",preds)
# np.savetxt("true_01_03.txt",true)

In [22]:
plt.figure(figsize=(10, 5))
plt.plot(preds)
plt.plot(true)
plt.show()

AttributeError: module 'matplotlib' has no attribute 'figure'

In [23]:
## 第三个测试样本
with torch.no_grad():
    mse_val = 0
    preds = []
    true = []
    alphas = []
    betas = []
    for batch_x, batch_y in test_loader_3:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        output, a, b = model(batch_x)
        output = output.squeeze(1)
        preds.append(output.detach().cpu().numpy())
        true.append(batch_y.detach().cpu().numpy())
        alphas.append(a.detach().cpu().numpy())
        betas.append(b.detach().cpu().numpy())
        mse_val += loss(output, batch_y).item() * batch_x.shape[0]
preds = np.concatenate(preds)
true = np.concatenate(true)

NameError: name 'test_loader_3' is not defined

In [ ]:
preds = (preds * (y_train_max - y_train_min) + y_train_min) * 0.2 + 0.8
true = ((true * (y_train_max - y_train_min) + y_train_min)) * 0.2 + 0.8
mae[3] = mean_absolute_error(true, preds) * 100
mape[3] = np.mean(np.abs((true - preds) / true)) * 100
rmse[3] = sqrt(mean_squared_error(true, preds)) * 100
R2[3] = r2_score(true, preds) * 100
mae[3], mape[3], rmse[3], R2[3]

In [24]:
# np.savetxt("preds_01_04.txt",preds)
# np.savetxt("true_01_04.txt",true)

In [25]:
plt.figure(figsize=(10, 5))
plt.plot(preds)
plt.plot(true)
plt.show()

AttributeError: module 'matplotlib' has no attribute 'figure'

In [26]:
## 第四个测试样本
with torch.no_grad():
    mse_val = 0
    preds = []
    true = []
    alphas = []
    betas = []
    for batch_x, batch_y in test_loader_4:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        output, a, b = model(batch_x)
        output = output.squeeze(1)
        preds.append(output.detach().cpu().numpy())
        true.append(batch_y.detach().cpu().numpy())
        alphas.append(a.detach().cpu().numpy())
        betas.append(b.detach().cpu().numpy())
        mse_val += loss(output, batch_y).item() * batch_x.shape[0]
preds = np.concatenate(preds)
true = np.concatenate(true)

NameError: name 'test_loader_4' is not defined

In [ ]:
preds = (preds * (y_train_max - y_train_min) + y_train_min) * 0.2 + 0.8
true = ((true * (y_train_max - y_train_min) + y_train_min)) * 0.2 + 0.8
mae[4] = mean_absolute_error(true, preds) * 100
mape[4] = np.mean(np.abs((true - preds) / true)) * 100
rmse[4] = sqrt(mean_squared_error(true, preds)) * 100
R2[4] = r2_score(true, preds) * 100
mae[4], mape[4], rmse[4], R2[4]

In [27]:
plt.figure(figsize=(10, 5))
plt.plot(preds)
plt.plot(true)
plt.show()

AttributeError: module 'matplotlib' has no attribute 'figure'

In [ ]:
## 第五个测试样本
with torch.no_grad():
    mse_val = 0
    preds = []
    true = []
    alphas = []
    betas = []
    for batch_x, batch_y in test_loader_5:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        output, a, b = model(batch_x)
        output = output.squeeze(1)
        preds.append(output.detach().cpu().numpy())
        true.append(batch_y.detach().cpu().numpy())
        alphas.append(a.detach().cpu().numpy())
        betas.append(b.detach().cpu().numpy())
        mse_val += loss(output, batch_y).item() * batch_x.shape[0]
preds = np.concatenate(preds)
true = np.concatenate(true)

In [28]:
preds = (preds * (y_train_max - y_train_min) + y_train_min) * 0.2 + 0.8
true = ((true * (y_train_max - y_train_min) + y_train_min)) * 0.2 + 0.8
mae[5] = mean_absolute_error(true, preds) * 100
mape[5] = np.mean(np.abs((true - preds) / true)) * 100
rmse[5] = sqrt(mean_squared_error(true, preds)) * 100
R2[5] = r2_score(true, preds) * 100
mae[5], mape[5], rmse[5], R2[5]

NameError: name 'y_train_max' is not defined

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(preds)
plt.plot(true)
plt.show()

In [29]:
## 第六个测试样本
with torch.no_grad():
    mse_val = 0
    preds = []
    true = []
    alphas = []
    betas = []
    for batch_x, batch_y in test_loader_5:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        output, a, b = model(batch_x)
        output = output.squeeze(1)
        preds.append(output.detach().cpu().numpy())
        true.append(batch_y.detach().cpu().numpy())
        alphas.append(a.detach().cpu().numpy())
        betas.append(b.detach().cpu().numpy())
        mse_val += loss(output, batch_y).item() * batch_x.shape[0]
preds = np.concatenate(preds)
true = np.concatenate(true)

NameError: name 'test_loader_5' is not defined

In [ ]:
preds = (preds * (y_train_max - y_train_min) + y_train_min) * 0.2 + 0.8
true = ((true * (y_train_max - y_train_min) + y_train_min)) * 0.2 + 0.8
mae[6] = mean_absolute_error(true, preds) * 100
mape[6] = np.mean(np.abs((true - preds) / true)) * 100
rmse[6] = sqrt(mean_squared_error(true, preds)) * 100
R2[6] = r2_score(true, preds) * 100
mae[6], mape[6], rmse[6], R2[6]

In [30]:
plt.figure(figsize=(10, 5))
plt.plot(preds)
plt.plot(true)
plt.show()

AttributeError: module 'matplotlib' has no attribute 'figure'

In [ ]:
Mean_mae = np.mean(mae)
Mean_mape = np.mean(mape)
Mean_rmse = np.mean(rmse)
Mean_R2 = np.mean(R2)
Mean_mae, Mean_mape, Mean_rmse, Mean_R2

In [31]:
alphas = np.concatenate(alphas)
betas = np.concatenate(betas)

ValueError: need at least one array to concatenate

In [32]:
alphas = alphas.mean(axis=0)
betas = betas.mean(axis=0)

AttributeError: 'list' object has no attribute 'mean'

In [33]:
alphas = alphas[..., 0]
betas = betas[..., 0]

TypeError: list indices must be integers or slices, not tuple

In [ ]:
alphas = alphas.transpose(1, 0)

In [34]:
# fig, ax = plt.subplots(figsize=(10, 10))
# im = ax.imshow(alphas)
# ax.set_xticks(np.arange(X_train_t.shape[1]))
# ax.set_yticks(np.arange(len(cols)))
# ax.set_xticklabels(["t-"+str(i) for i in np.arange(X_train_t.shape[1], -1, -1)])
# ax.set_yticklabels(list(cols))
# for i in range(len(cols)):
#     for j in range(X_train_t.shape[1]):
#         text = ax.text(j, i, round(alphas[i, j], 3),
#                        ha="center", va="center", color="w")
# ax.set_title("Importance of features and timesteps")
# #fig.tight_layout()
# plt.show()

In [35]:
plt.figure(figsize=(5, 2.5))
plt.title("Feature importance")
plt.bar(range(len(cols)), betas)
plt.xticks(ticks=range(len(cols)), labels=cols, rotation=90)

AttributeError: module 'matplotlib' has no attribute 'figure'

In [ ]:
print(betas)

In [ ]:
Sorted_betas = sorted(betas, reverse=True)
print(Sorted_betas[:6])

In [36]:
betaCurrent = sum(betas[0:6])
betaVoltage = sum(betas[6:12])
betaTemperature = sum(betas[12:18])
betaCategory = np.array([betaCurrent, betaVoltage, betaTemperature])
plt.figure(figsize=(5, 2.5))
plt.title("Category Feature importance")
plt.bar(range(len(betaCategory)), betaCategory)
plt.xticks(ticks=range(len(betaCategory)), rotation=90)

AttributeError: module 'matplotlib' has no attribute 'figure'

In [ ]:
betaCharge_1A = betas[0] + betas[6] + betas[12]
betaCharge_3V6 = betas[1] + betas[7] + betas[13]
betaCharge_20SOC = betas[2] + betas[8] + betas[14]
betaDischarge_20SOC = betas[3] + betas[9] + betas[15]
betaDischarge_80SOC = betas[4] + betas[10] + betas[16]
betaDischarge_20to80SOC = betas[5] + betas[11] + betas[17]
betaStage = np.array([betaCharge_1A, betaCharge_3V6, betaCharge_20SOC, betaDischarge_20SOC, betaDischarge_80SOC,
                      betaDischarge_20to80SOC])
plt.figure(figsize=(5, 2.5))
plt.title("Stage Feature importance")
plt.bar(range(len(betaStage)), betaStage)
plt.xticks(ticks=range(len(betaStage)), rotation=90)